# Eye of the Emergency

Les plateformes de médias sociaux sont aujourd’hui devenues une partie intégrante de
nos vies, nous fournissant des informations en temps réel sur les événements qui se
produisent autour de nous. Toutefois, cet essor s'est accompagné d'une augmentation
de la désinformation et des "fake news". En cas de catastrophe ou de crise, ces fausses
informations peuvent être particulièrement dangereuses, car elles peuvent entraver les
efforts d'intervention d'urgence et provoquer la panique au sein de la population.